In [1]:
import pickle
import torch
import torchaudio
import time
from audiocraft.models import MusicGen
from audiocraft.data.audio import audio_write
from audiocraft.data.audio_utils import convert_audio
from audiocraft.modules.conditioners import ConditioningAttributes

objc[56374]: Class AVFFrameReceiver is implemented in both /Users/maxj/miniconda3/envs/audiocraft/lib/libavdevice.58.8.100.dylib (0x10fdd4798) and /Users/maxj/miniconda3/envs/audiocraft/lib/python3.9/site-packages/av/.dylibs/libavdevice.59.7.100.dylib (0x11f818778). One of the two will be used. Which one is undefined.
objc[56374]: Class AVFAudioReceiver is implemented in both /Users/maxj/miniconda3/envs/audiocraft/lib/libavdevice.58.8.100.dylib (0x10fdd47e8) and /Users/maxj/miniconda3/envs/audiocraft/lib/python3.9/site-packages/av/.dylibs/libavdevice.59.7.100.dylib (0x11f8187c8). One of the two will be used. Which one is undefined.


In [2]:
model = MusicGen.get_pretrained('facebook/musicgen-medium')

/Users/maxj/miniconda3/envs/audiocraft/lib/python3.9/site-packages/torch/nn/utils/weight_norm.py:30: UserWarning: torch.nn.utils.weight_norm is deprecated in favor of torch.nn.utils.parametrizations.weight_norm.
  warnings.warn("torch.nn.utils.weight_norm is deprecated in favor of torch.nn.utils.parametrizations.weight_norm.")


In [4]:
model.lm

LMModel(
  (cfg_dropout): ClassifierFreeGuidanceDropout(p=0.3)
  (att_dropout): AttributeDropout({})
  (condition_provider): ConditioningProvider(
    (conditioners): ModuleDict(
      (description): T5Conditioner(
        (output_proj): Linear(in_features=768, out_features=1536, bias=True)
      )
    )
  )
  (fuser): ConditionFuser()
  (emb): ModuleList(
    (0-3): 4 x ScaledEmbedding(2049, 1536)
  )
  (transformer): StreamingTransformer(
    (layers): ModuleList(
      (0-47): 48 x StreamingTransformerLayer(
        (self_attn): StreamingMultiheadAttention(
          (out_proj): Linear(in_features=1536, out_features=1536, bias=False)
        )
        (linear1): Linear(in_features=1536, out_features=6144, bias=False)
        (dropout): Dropout(p=0.0, inplace=False)
        (linear2): Linear(in_features=6144, out_features=1536, bias=False)
        (norm1): LayerNorm((1536,), eps=1e-05, elementwise_affine=True)
        (norm2): LayerNorm((1536,), eps=1e-05, elementwise_affine=True)
  

In [5]:
model.compression_model

EncodecModel(
  (encoder): SEANetEncoder(
    (model): Sequential(
      (0): StreamableConv1d(
        (conv): NormConv1d(
          (conv): Conv1d(1, 64, kernel_size=(7,), stride=(1,))
          (norm): Identity()
        )
      )
      (1): SEANetResnetBlock(
        (block): Sequential(
          (0): ELU(alpha=1.0)
          (1): StreamableConv1d(
            (conv): NormConv1d(
              (conv): Conv1d(64, 32, kernel_size=(3,), stride=(1,))
              (norm): Identity()
            )
          )
          (2): ELU(alpha=1.0)
          (3): StreamableConv1d(
            (conv): NormConv1d(
              (conv): Conv1d(32, 64, kernel_size=(1,), stride=(1,))
              (norm): Identity()
            )
          )
        )
        (shortcut): Identity()
      )
      (2): ELU(alpha=1.0)
      (3): StreamableConv1d(
        (conv): NormConv1d(
          (conv): Conv1d(64, 128, kernel_size=(8,), stride=(4,))
          (norm): Identity()
        )
      )
      (4): SEANetRe

In [6]:
def compute_file_embedding(filename):
    return _embed_codes(filename, _load_input(filename))

def _load_input(filename):
    t1 = time.perf_counter()
    wav, sr = torchaudio.load(filename)
    duration = wav.shape[1] / sr
    if duration > 25 * 60:
        raise ValueError(f"track is too long: {duration}s")
    wav = convert_audio(wav, sr, model.sample_rate, model.audio_channels)
    wav = wav.unsqueeze(0)
    t2 = time.perf_counter()
    print(filename, "load", t2 - t1)
    with torch.no_grad():
        encoded, _ = model.compression_model.encode(wav)
    t3 = time.perf_counter()
    print(filename, "encode", t3-t2)
    return encoded
    
def _embed_codes(filename, encoded):
    t1 = time.perf_counter()
    context_limit = model.max_duration * model.frame_rate - 1
    emb = model.lm.compute_sequence_embedding(encoded[0], context_limit)
    t2 = time.perf_counter()
    print(filename, "embed", t2-t1)
    return emb

In [8]:
toks = _load_input("/Users/maxj/Music/Rips/Spotify/AIFF/2fox - We Are - 6Lxi9O5CWRgjrqvWdbn2lT.aiff")

/Users/maxj/Music/Rips/Spotify/AIFF/2fox - We Are - 6Lxi9O5CWRgjrqvWdbn2lT.aiff load 0.24382141699999238
/Users/maxj/Music/Rips/Spotify/AIFF/2fox - We Are - 6Lxi9O5CWRgjrqvWdbn2lT.aiff encode 20.495131500000014


In [13]:
toks

tensor([[[1077, 1273, 1607,  ...,   83,   83,   83],
         [ 845, 1642,   21,  ..., 2044, 2044, 2044],
         [1872,  373,  650,  ..., 2019, 2019, 2019],
         [1518, 1696, 1284,  ..., 1770, 1770, 1770]]])

In [14]:
toks.max()

tensor(2047)